In [ ]:
#| default_exp generative_basics

In [ ]:
#|hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#|hide
from nbdev.showdoc import show_doc

In [ ]:
#|export
from fastai.text.all import *
from fastinference.inference import *

In [ ]:
#|export
class MolSampler():
    """
    Base class for molecule generation
    """
    def __init__(self,model_fname:str,text:str='',cpu:bool=False):      
        
        self.model = load_learner(model_fname, cpu=cpu)
        self.text = text
    

    def base_sampler(self, max_size:int=100, temperature:float=1.0, **kwargs):
        
        """Base sampler to generate one SMILES using a chemistry model trained with fastai
        
        
        Arguments:
        
        temperature : float
            Sampling temperature (default = 1.0)
        
        max_size : int
            Maximum size of the SMILES strings (default = 100)
        
        
        Returns:
        
        decoded : str
            A string representing a SMILES of a generated molecule
        
        """
        
        act = getattr(self.model.loss_func, 'activation', noop)
        self.model.model.reset()    # Reset the model
        stop_index = self.model.dls.train.vocab.index(BOS)        # Define the stop token
        idxs = self.model.dls.test_dl([self.text]).items[0].to(x.model.dls.device)
        nums = self.model.dls.train_ds.numericalize     # Numericalize (used to decode)
        accum_idxs = []                   # Store predicted tokens
        self.model.model.eval()

        for _ in range(max_size):
            with torch.no_grad(): preds = self.model.model(idxs[None])[0][-1]

            res = act(preds)
            if temperature != 1.: res.pow_(1 / temperature)
            idx = torch.multinomial(res, 1).item()
            if idx != stop_index:
                accum_idxs.append(idx)
                idxs = TensorText(idxs.new_tensor([idx]))
            else:
                break
        decoded = ''.join([nums.vocab[o] for o in accum_idxs if nums.vocab[o] not in [BOS, PAD]])  # Decode predicted tokens
        return decoded
    
    def generate_mols(self, max_size, max_mols, temperature,**kwargs):
        '''Generate molecules using a base sampler'''

        return [self.base_sampler(max_size=max_size, temperature=temperature) for i in range(max_mols)]

In [ ]:
show_doc(MolSampler)

---

### MolSampler

>      MolSampler (model_fname:str, text:str='', cpu:bool=False)

Base class for molecule generation

In [ ]:
show_doc(MolSampler.base_sampler)

---

#### MolSampler.base_sampler

>      MolSampler.base_sampler (max_size:int=100, temperature:float=1.0,
>                               **kwargs)

Base sampler to generate one SMILES using a chemistry model trained with fastai

In [ ]:
show_doc(MolSampler.generate_mols)

---

#### MolSampler.generate_mols

>      MolSampler.generate_mols (max_size, max_mols, temperature, **kwargs)

Generate molecules using a base sampler

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()